Moved to Jupyter notebook for Challenge 3, just expecting to keep hitting the rate_limit otherwise

In [108]:
# DEPENDENCIES
from keyring import github_token
import os
# print(os.getcwd())
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('display.max_columns', None)

github_session = requests.Session()
github_session.auth = ('SamPoad', github_token)



# RESOURCES
# https://developer.github.com/v3/search/#search-repositories
# up to 1,000 results per search
# https://developer.github.com/v3/search/#constructing-a-search-query



In [16]:
rate_message = (requests.get('https://api.github.com/rate_limit')).json()
print(rate_message)

{'resources': {'core': {'limit': 60, 'remaining': 0, 'reset': 1569974903}, 'search': {'limit': 10, 'remaining': 10, 'reset': 1569972165}, 'graphql': {'limit': 0, 'remaining': 0, 'reset': 1569975705}, 'integration_manifest': {'limit': 5000, 'remaining': 5000, 'reset': 1569975705}}, 'rate': {'limit': 60, 'remaining': 0, 'reset': 1569974903}}


In [2]:
# check to make sure I've got the right search query
# url = 'https://api.github.com/search/repositories?q=stars:>=10000'
# url = 'https://api.github.com/search/code?q=filename:scavenger+repo:ironhack-labs/data-labs'
# url = 'https://api.github.com/search/code?q=filename:scavenger+org:ironhack-labs'
# q=repo:ironhack-labs/data-labs/+filename:.scavengerhunt # not found
# q=filename:.scavengerhunt+repo:ironhack-labs/data-labs/ # not found
# q=repo:ironhack-labs/data-labs/+filename:scavengerhunt # not found
# q=filename:scavengerhunt+in:repo:ironhack-labs/data-labs # not found
# q=filename:scavengerhunt+repo:ironhack-labs/data-labs # not found
# q=org:Ironhack-labs+repo:ironhack-labs/data-labs # not found
# from terminal, attempted
# curl -u SamPoad:"my token string here" "https://api.github.com/Search/repositories/q=repo:ironhack-labs/data-labs"
# looks like the below is the better format to use?
# https://api.github.com/Search/repositories?q=repo:ironhack-labs/data-labs
# and still doesn't work
# at least I know it CAN be done
# https://api.github.com/search/repositories?q=stars:>=10000+language:go&sort=stars&order=desc
# url = 'https://api.github.com/Search/repositories?q=scavengerhunt'
# not even the above works
# FINALLY found this file
# https://api.github.com/search/code?q=scavenger+repo:ironhack-labs/data-labs
# url = 'https://api.github.com/search/code?q=scavenger+repo:ironhack-labs/data-labs
# https://developer.github.com/v3/repos/contents/#get-contents
# https://api.github.com/search/repositories?q=repo:ironhack-labs/data-labs
# https://github.com/ironhack-datalabs/scavenger
# ^^^ This is the repo I should be looking in per the ReadMe, not ironhack-labs/data-labs.
# GET /repos/:owner/:repo/contents/:path
# https://api.github.com/repos/ironhack-datalabs/scavenger/contents/
# ^^^ this seems to work. need to iterate through the subfolders?
# https://api.github.com/repos/ironhack-datalabs/scavenger/contents/15024 is a subfolder,
#     so for folder in url/contents/:
#         if the filename contains 'scavenger':
#             add it to a list
# do the above with a nested for loop? 

# Ran the following in terminal so I could see the number of pages
# curl -I "https://api.github.com/repos/ironhack-datalabs/scavenger/contents"
# this returned without any pagination, which makes sense. the default response is 30 entries, and there's only 25 folders




# Nested For Loop, actually doesn't need to be nested it I just pull out the value from key:value of name:foldername
contents_url = 'https://api.github.com/repos/ironhack-datalabs/scavenger/contents'
foldernames = []
response = requests.get(contents_url)
result = response.json()
folder_dict = pd.DataFrame(result)
print(folder_dict['name'])


0     .gitignore
1          15024
2          15534
3          17020
4          30351
5          40303
6          44639
7          45525
8          47222
9          47830
10         49418
11         50896
12         55417
13         55685
14         60224
15         64880
16         66032
17         68848
18         70751
19         70985
20         88596
21         89046
22         89338
23         91701
24         97881
25         98750
Name: name, dtype: object


In [18]:
for i in folder_dict['name']: # for each folder in the scavenger repo
    sub_url = f'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{i}' # use the api to open that folder
    for file in sub_url: #for each file within the sub-url (NOPE determined this is reading as "for each character in the URL string")
        sub_url2 = f'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{i}/{file}'
        response2 = requests.get(sub_url2) # if that filename has "scavenger" in it
        result2 = response2.json()   
        scav_files_df = pd.DataFrame(result) # throw it in a dataframe



                                              _links  \
0  {'self': 'https://api.github.com/repos/ironhac...   
1  {'self': 'https://api.github.com/repos/ironhac...   
2  {'self': 'https://api.github.com/repos/ironhac...   
3  {'self': 'https://api.github.com/repos/ironhac...   
4  {'self': 'https://api.github.com/repos/ironhac...   

                                        download_url  \
0  https://raw.githubusercontent.com/ironhack-dat...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                             git_url  \
0  https://api.github.com/repos/ironhack-datalabs...   
1  https://api.github.com/repos/ironhack-datalabs...   
2  https://api.github.com/repos/ironhack-datalabs...   
3  https://api.github.com/repos/ironhack-datalabs...   
4  https://api.github.com/repos/ironhack-datal

In [22]:
# print(scav_files_df.head())
print(scav_files_df.dtypes)

_links          object
download_url    object
git_url         object
html_url        object
name            object
path            object
sha             object
size             int64
type            object
url             object
dtype: object


So we see from above that there is a download_url in the dataframe.
probably my best bet to reading the files themselves.
look up previous labwork that covered reading text


In [24]:
print(scav_files_df['download_url'][0]) # or this is just the .gitignore file.

https://raw.githubusercontent.com/ironhack-datalabs/scavenger/master/.gitignore


In [26]:
print(scav_files_df['_links'][1])

{'self': 'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/15024?ref=master', 'git': 'https://api.github.com/repos/ironhack-datalabs/scavenger/git/trees/2945e51c87ad5da893c954afcf092f06343bbb7d', 'html': 'https://github.com/ironhack-datalabs/scavenger/tree/master/15024'}


In [3]:
for folder in folder_dict['name']: # for each folder in the scavenger repo
    sub_url = f'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{folder}' # use the api to open that folder
    for file in requests.get(sub_url): #for each file within the sub-url
        sub_url2 = f'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{folder}/{file}'
#         print(file["name"]) returns as a byte
        response2 = requests.get(sub_url2) # if that filename has "scavenger" in it
        result2 = response2.json()
        flat_result2 = json_normalize(result2)
#         scav_files_df = pd.DataFrame(result2) # throw it in a dataframe
print(flat_result2)
# this method results in the API rate limit getting hit. need to refactor.
# try and instead only Call/grab the files with scavenger

                                documentation_url  \
0  https://developer.github.com/v3/#rate-limiting   

                                             message  
0  API rate limit exceeded for 73.138.93.245. (Bu...  


In [32]:
print(scav_files_df.dtypes)
# print(scav_files_df['name'])

0     .gitignore
1          15024
2          15534
3          17020
4          30351
5          40303
6          44639
7          45525
8          47222
9          47830
10         49418
11         50896
12         55417
13         55685
14         60224
15         64880
16         66032
17         68848
18         70751
19         70985
20         88596
21         89046
22         89338
23         91701
24         97881
25         98750
Name: name, dtype: object


In [ ]:
# Scrapped this
# to try and pull in all the files? 
for i in range(.0001, .0024):
    url = f'https://api.github.com/Search/repositories/q=repo:ironhack-labs/data-labs/filename:{i}.scavengerhunt'
    response = requests.get(url)
    result = pd.read_json(response.json())
    result2.append(result)
print(result2)

so far, I have that I need to loop through 
the raw.github url like this one
https://raw.githubusercontent.com/ironhack-datalabs/scavenger/master/15024/.0006.scavengerhunt
so that I can have the system READ and store the data of the scavengerhunt files.

i'm stuck on getting to that point, still held up on using the API to get exactly what I want as a dictionary.
latest attempt is to flatten it for easier retrieval of the raw.github URLs

In [12]:
# in class, working with Ani
import os
cwd = os.getcwd()
print(cwd)
output_df = pd.read_json("output.json")
# print(output_df.head())
# data = pd.DataFrame(response.json())



/Users/sampoad/Documents/Ironhack/data-labs/module-1/lab-api-scavenger-game/your-code
['.gitignore', '15024', '15534', '17020', '30351', '40303', '44639', '45525', '47222', '47830', '49418', '50896', '55417', '55685', '60224', '64880', '66032', '68848', '70751', '70985', '88596', '89046', '89338', '91701', '97881', '98750']


In [68]:
repo_files = pd.DataFrame(columns=flattened_data.columns)

In [69]:
flattened_data

,_links.git,_links.html,_links.self,content,download_url,encoding,git_url,html_url,name,path,sha,size,type,url
0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,LkRTX1N0b3JlCg==\n,https://raw.githubusercontent.com/ironhack-dat...,base64,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,.gitignore,.gitignore,e43b0f988953ae3a84b00331d0ccf5f7d51cb3cf,10,file,https://api.github.com/repos/ironhack-datalabs...


In [72]:
pd.concat([repo_files,flattened_data])

,_links.git,_links.html,_links.self,content,download_url,encoding,git_url,html_url,name,path,sha,size,type,url
0,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,https://api.github.com/repos/ironhack-datalabs...,LkRTX1N0b3JlCg==\n,https://raw.githubusercontent.com/ironhack-dat...,base64,https://api.github.com/repos/ironhack-datalabs...,https://github.com/ironhack-datalabs/scavenger...,.gitignore,.gitignore,e43b0f988953ae3a84b00331d0ccf5f7d51cb3cf,10,file,https://api.github.com/repos/ironhack-datalabs...


In [100]:
'''
does this need to be flattened?
for each folder in name column:
    read the filenames in the folder
    if scavengerhunt is in filename:
        read the file
        append the contents to a list
        
eventually concat the list
'''

# print(list(output_df['name']))

# repo_files = pd.DataFrame(columns=flattened_data.columns)
repo_files = []
for foldername in output_df['name']:
    url = f'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{foldername}'
    response = requests.get(url, auth = github_session.auth)
    result = response.json()
    flattened_data = json_normalize(result)
    repo_files.append(flattened_data)

repo_df = pd.concat(repo_files, sort=False)
print(repo_df.head())






'''
subset the results dataframe (after turning it into a dataframe)
with only the filenames that have 'scavengerhunt' in them

then call the api to retrieve the Contents of those individual ~25 files,
append it to a list,
concat that list
'''
# unless we can figure out how to call the API and say for each folder give me all the files in a repo.

# flattened_data

                                          _links.git  \
0  https://api.github.com/repos/ironhack-datalabs...   
0  https://api.github.com/repos/ironhack-datalabs...   
1  https://api.github.com/repos/ironhack-datalabs...   
2  https://api.github.com/repos/ironhack-datalabs...   
3  https://api.github.com/repos/ironhack-datalabs...   

                                         _links.html  \
0  https://github.com/ironhack-datalabs/scavenger...   
0  https://github.com/ironhack-datalabs/scavenger...   
1  https://github.com/ironhack-datalabs/scavenger...   
2  https://github.com/ironhack-datalabs/scavenger...   
3  https://github.com/ironhack-datalabs/scavenger...   

                                         _links.self             content  \
0  https://api.github.com/repos/ironhack-datalabs...  LkRTX1N0b3JlCg==\n   
0  https://api.github.com/repos/ironhack-datalabs...                 NaN   
1  https://api.github.com/repos/ironhack-datalabs...                 NaN   
2  https://api.github.

"\nsubset the results dataframe (after turning it into a dataframe)\nwith only the filenames that have 'scavengerhunt' in them\n\nthen call the api to retrieve the Contents of those individual ~25 files,\nappend it to a list,\nconcat that list\n"

In [104]:
# repo_df = pd.DataFrame(repo_files, )
# flat_repo = json_normalize(repo_files)
# print(flat_repo)
# add the flat repo folders into a dataframe
# subset that dataframe to just the folders

# pd.set_option('display.max_columns', None)
# print(repo_df)
# repo_df = repo_files[
#     repo_files['name'] == '*.scavengerhunt'
# ]
# print(repo_df)

repo_df2 = repo_df[repo_df['size'] > 0]

repo_df3 = repo_df2[repo_df2['name'] != '.gitignore']

print(repo_df3.head())


# repo_df3 = pd.DataFrame(columns=repo_df2.columns)
# for row in repo_df2['name']:
# #     if row.contains('scavengerhunt'):
#     if '.scavengerhunt' in row:
#         repo_df3 = pd.concat([repo_df3, repo_df2['name']])

# # print(repo_df2)
# print(repo_df3)

# a['Names'].str.contains('Mel')






                                          _links.git  \
0  https://api.github.com/repos/ironhack-datalabs...   
0  https://api.github.com/repos/ironhack-datalabs...   
1  https://api.github.com/repos/ironhack-datalabs...   
0  https://api.github.com/repos/ironhack-datalabs...   
0  https://api.github.com/repos/ironhack-datalabs...   

                                         _links.html  \
0  https://github.com/ironhack-datalabs/scavenger...   
0  https://github.com/ironhack-datalabs/scavenger...   
1  https://github.com/ironhack-datalabs/scavenger...   
0  https://github.com/ironhack-datalabs/scavenger...   
0  https://github.com/ironhack-datalabs/scavenger...   

                                         _links.self content  \
0  https://api.github.com/repos/ironhack-datalabs...     NaN   
0  https://api.github.com/repos/ironhack-datalabs...     NaN   
1  https://api.github.com/repos/ironhack-datalabs...     NaN   
0  https://api.github.com/repos/ironhack-datalabs...     NaN   
0  htt

In [99]:
# scav_text = []
# for foldername in repo_df2['name']:
#     for number in [.0001, .0002, .0003]:
#         url = f'https://github.com/repos/ironhack-datalabs/scavenger/blob/master/{foldername}/{number}.scavengerhunt/'
#         response = pd.read_json(url)
#         scav_text.append(response)
# print(scav_text)


In [ ]:
# scav_text = []
# for foldername in output_df['name']:
#     for filename in repo_df2['name']:
#         url = f'https://api.github.com/repos/ironhack-datalabs/scavenger/blob/master/{foldername}/{filename}.scavengerhunt/'
#         response = pd.read_json(url)
#         scav_text.append(response)

In [ ]:
# https://raw.githubusercontent.com/ironhack-datalabs/scavenger/master/15024/.0006.scavengerhunt



In [105]:
print(list(repo_df3))

['_links.git', '_links.html', '_links.self', 'content', 'download_url', 'encoding', 'git_url', 'html_url', 'name', 'path', 'sha', 'size', 'type', 'url']


In [126]:
# print(repo_df3['download_url'])

import base64

scav_text = []
for url in repo_df3['download_url']:
    response = requests.get(url)
    print(response)
#     base64.b64decode(s[, altchars])
# read the file
    readurl = pd.read_json(response)
    response2 = base64.b64decode(response, 'utf-8')
    result = response2.json()
    print(result)
    scav_text.append(result)
print(scav_text)   



<Response [200]>


ValueError: Invalid file path or buffer object type: <class 'requests.models.Response'>